<a href="https://colab.research.google.com/github/Haebuk/kuggle/blob/main/gamestop_sweep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install wandb -qqq

     |████████████████████████████████| 1.6 MB 8.5 MB/s 
     |████████████████████████████████| 133 kB 51.5 MB/s 
     |████████████████████████████████| 97 kB 8.8 MB/s 
     |████████████████████████████████| 170 kB 58.3 MB/s 
     |████████████████████████████████| 63 kB 2.2 MB/s 


In [2]:
!wandb login

wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [3]:
import wandb 
from wandb.keras import WandbCallback
import tensorflow as tf
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from sklearn.model_selection import train_test_split

PATH = '/content/drive/MyDrive/input/'
df = pd.read_csv(PATH + 'GME_scaled.csv')

default_configs = {
    'learning_rate': 0.001,
    'dropout_rate1': 0.2,
    'dropout_rate2': 0.2,
    'hidden1': 50,
    'hidden2': 50,
    'time_step': 60,
}
wandb.init(project='gamestop_prediction', entity='kuggle', config=default_configs)
config = wandb.config

X = df.drop('date', axis=1).values
y = df['open_price'].values
X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.1, shuffle=False)

time_step = config.time_step
X_train, y_train, X_test, y_test = [], [], [], []
for i in range(time_step,len(X_tr)): # train
    X_train.append(X_tr[i-time_step:i,:])
    y_train.append(y_tr[i])
for i in range(time_step, len(X_te)): # test
    X_test.append(X_te[i-time_step:i,:])
    y_test.append(y_te[i])
X_train, y_train = np.array(X_train), np.array(y_train)
X_test, y_test = np.array(X_test), np.array(y_test)
# LSTM input shape 조건(batch_size, timestep, feature_num)에 맞게 reshape
X_train = np.reshape(X_train, (-1,X_train.shape[1],2))
X_test = np.reshape(X_test, (-1,X_test.shape[1],2))

model = tf.keras.Sequential([
LSTM(config.hidden1, return_sequences=True, input_shape=(config.time_step, 2)),
Dropout(config.dropout_rate1),
LSTM(config.hidden2, return_sequences=True),
Dropout(config.dropout_rate2),
Dense(1),
])

opt = tf.keras.optimizers.Adam(learning_rate=config.learning_rate)

early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

model.compile(optimizer=opt,
            loss='mse',
            metrics=['mse'])

model.fit(X_train, y_train,
        validation_data=(X_test, y_test),
        epochs=200, callbacks=[WandbCallback(), early_stopping])

wandb: Currently logged in as: kade (use `wandb login --relogin` to force relogin)


Epoch 1/200
81/81 [==============================] - 9s 22ms/step - loss: 1.1477 - mse: 1.1477 - val_loss: 1.9787 - val_mse: 1.9787


wandb: ERROR Can't save model, h5py returned error: Layer ModuleWrapper has arguments in `__init__` and therefore must override `get_config`.


Epoch 2/200
81/81 [==============================] - 1s 11ms/step - loss: 0.1779 - mse: 0.1779 - val_loss: 1.1785 - val_mse: 1.1785
Epoch 3/200
81/81 [==============================] - 1s 11ms/step - loss: 0.1022 - mse: 0.1022 - val_loss: 1.3924 - val_mse: 1.3924
Epoch 4/200
81/81 [==============================] - 1s 11ms/step - loss: 0.0797 - mse: 0.0797 - val_loss: 1.0705 - val_mse: 1.0705
Epoch 5/200
81/81 [==============================] - 1s 11ms/step - loss: 0.0829 - mse: 0.0829 - val_loss: 1.1751 - val_mse: 1.1751
Epoch 6/200
81/81 [==============================] - 1s 11ms/step - loss: 0.0728 - mse: 0.0728 - val_loss: 1.0129 - val_mse: 1.0129
Epoch 7/200
81/81 [==============================] - 1s 11ms/step - loss: 0.0785 - mse: 0.0785 - val_loss: 1.1800 - val_mse: 1.1800
Epoch 8/200
81/81 [==============================] - 1s 11ms/step - loss: 0.0638 - mse: 0.0638 - val_loss: 0.8345 - val_mse: 0.8345
Epoch 9/200
81/81 [==============================] - 1s 11ms/step - loss: 0.

In [4]:
with open('train.py', 'w') as f:
    f.write("""
import wandb 
from wandb.keras import WandbCallback
import tensorflow as tf
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from sklearn.model_selection import train_test_split

PATH = '/content/drive/MyDrive/input/'
df = pd.read_csv(PATH + 'GME_scaled.csv')

default_configs = {
    'learning_rate': 0.001,
    'dropout_rate1': 0.2,
    'dropout_rate2': 0.2,
    'dropout_rate3': 0.2,
    'hidden1': 50,
    'hidden2': 50,
    'hidden3': 50,
    'time_step': 60,
}
wandb.init(project='gamestop_prediction', config=default_configs, magic=True)
config = wandb.config

X = df.drop('date', axis=1).values
y = df['open_price'].values
X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.1, shuffle=False)

time_step = config.time_step
X_train, y_train, X_test, y_test = [], [], [], []
for i in range(time_step,len(X_tr)): # train
    X_train.append(X_tr[i-time_step:i,:])
    y_train.append(y_tr[i])
for i in range(time_step, len(X_te)): # test
    X_test.append(X_te[i-time_step:i,:])
    y_test.append(y_te[i])
X_train, y_train = np.array(X_train), np.array(y_train)
X_test, y_test = np.array(X_test), np.array(y_test)
# LSTM input shape 조건(batch_size, timestep, feature_num)에 맞게 reshape
X_train = np.reshape(X_train, (-1,X_train.shape[1],2))
X_test = np.reshape(X_test, (-1,X_test.shape[1],2))

model = tf.keras.Sequential([
LSTM(config.hidden1, return_sequences=True, input_shape=(config.time_step, 2)),
Dropout(config.dropout_rate1),
LSTM(config.hidden2, return_sequences=True),
Dropout(config.dropout_rate2),
LSTM(config.hidden3),
Dropout(config.dropout_rate3),
Dense(1),
])

opt = tf.keras.optimizers.Adam(learning_rate=config.learning_rate)

early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

model.compile(optimizer=opt,
            loss='mse',
            metrics=['mse'])

model.fit(X_train, y_train,
        validation_data=(X_test, y_test),
        epochs=200, callbacks=[WandbCallback(), early_stopping])
    """)

In [5]:
!head train.py


import wandb 
from wandb.keras import WandbCallback
import tensorflow as tf
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from sklearn.model_selection import train_test_split



In [ ]:
!wandb agent kuggle/gamestop_prediction/1ng3wx9s

wandb: Starting wandb agent 🕵️
2021-08-20 12:49:24,429 - wandb.wandb_agent - INFO - Running runs: []
2021-08-20 12:49:24,601 - wandb.wandb_agent - INFO - Agent received command: run
2021-08-20 12:49:24,601 - wandb.wandb_agent - INFO - Agent starting run with config:
	dropout_rate1: 0.44896520548391455
	dropout_rate2: 0.5276440928019034
	dropout_rate3: 0.5344824982636621
	hidden1: 72
	hidden2: 12
	hidden3: 13
	learning_rate: 0.04783929267673129
	time_step: 77
2021-08-20 12:49:24,603 - wandb.wandb_agent - INFO - About to run command: /usr/bin/env python train.py --dropout_rate1=0.44896520548391455 --dropout_rate2=0.5276440928019034 --dropout_rate3=0.5344824982636621 --hidden1=72 --hidden2=12 --hidden3=13 --learning_rate=0.04783929267673129 --time_step=77
wandb: Currently logged in as: kade (use `wandb login --relogin` to force relogin)
wandb: WARNING Ignored wandb.init() arg project when running a sweep
wandb: Tracking run with wandb version 0.12.0
wandb: Syncing run devoted-sweep-1
wand